# Goal is to display all workouts
#### * Query database
#### * Transform models into viewmodels
#### * Display data to user

In [30]:
import sqlite3
import json

In [13]:
DATABASE = 'workout.db'

### Define function to query database

In [14]:
def query_db(query, args=(), one=False):
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    cur = conn.execute(query, args)
    rv = cur.fetchall()
    conn.commit()
    cur.close()
    return (rv[0] if rv else None) if one else rv

### Query all workouts & topsets completed by Gabe(PersonId=1)

In [83]:
topsets = query_db("""
    SELECT 
        P.PersonId, 
        P.Name AS PersonName, 
        W.WorkoutId, 
        W.StartDate, 
        T.TopSetId, 
        E.ExcerciseId, 
        E.Name AS ExerciseName, 
        T.Repetitions, 
        T.Weight
    FROM Person P
         LEFT JOIN Workout W ON P.PersonId=W.PersonId
         LEFT JOIN TopSet T ON W.WorkoutId=T.WorkoutId
         LEFT JOIN Excercise E ON T.ExcerciseId=E.ExcerciseId
    WHERE P.PersonId=?""", [1])

for topset in topsets:
    print(f'WorkoutId: {topset["WorkoutId"]}, PersonId: {topset["PersonId"]} Person:{topset["PersonName"]}, StartDate: {topset["StartDate"]}, TopSetId: {topset["TopSetId"]}, ExcerciseId: {topset["ExcerciseId"]}, Exercise: {topset["ExerciseName"]}, Repetitions: {topset["Repetitions"]}, Weight: {topset["Weight"]}kg')

WorkoutId: 1, PersonId: 1 Person:Gabe, StartDate: 2022-06-29 00:00:00.000, TopSetId: 1, ExcerciseId: 2, Exercise: Bench, Repetitions: 11, Weight: 40kg
WorkoutId: 2, PersonId: 1 Person:Gabe, StartDate: 2022-07-07 00:00:00.000, TopSetId: 2, ExcerciseId: 1, Exercise: Squat, Repetitions: 5, Weight: 65kg
WorkoutId: 2, PersonId: 1 Person:Gabe, StartDate: 2022-07-07 00:00:00.000, TopSetId: 3, ExcerciseId: 4, Exercise: Hotep, Repetitions: 6, Weight: 30kg
WorkoutId: 3, PersonId: 1 Person:Gabe, StartDate: 2022-07-12 00:00:00.000, TopSetId: 4, ExcerciseId: 2, Exercise: Bench, Repetitions: 4, Weight: 60kg
WorkoutId: 3, PersonId: 1 Person:Gabe, StartDate: 2022-07-12 00:00:00.000, TopSetId: 5, ExcerciseId: 3, Exercise: Deadlift, Repetitions: 9, Weight: 100kg


### Transform entity to view model with the following schema
<pre>
{   "PersonId": 1,
    "PersonName": "Gabe",  
    "Exercises": [{"ExerciseIs": 1, "ExerciseName": "Squat"}, {"ExerciseIs": 2, "ExerciseName": "Bench"}]  
    "Workouts": [  
        {  
            "TopSetId": 1,
            "StartDate": "2022-06-29 00:00:00.000",  
            "TopSets": [  
                {  
                    "ExerciseId": 1,
                    "ExerciseName": "Squat",  
                    "Weight": 80,  
                    "Repetitions": 8  
                },  
                {  
                    "ExerciseId": 2,
                    "ExerciseName": "Bench",  
                    "Weight": 60,  
                    "Repetitions": 4  
                }  
            ]  
        },  
        {
            "TopSetId": 2,
            "StartDate": "2022-07-01 00:00:00.000",  
            "TopSets": [  
                {  
                    "ExerciseId": 1,
                    "ExerciseName": "Squat",  
                    "Weight": 85,  
                    "Repetitions": 4  
                },  
                {  
                    "ExerciseId": 2,
                    "ExerciseName": "Bench",  
                    "Weight": 65,  
                    "Repetitions": 5  
                }  
            ]  
        }  
    ]  
} 
</pre>

#### 1. Extract name from rows

In [37]:
def get_name(topsets):
    return topsets[0]['PersonName']

get_name(topsets)

'Gabe'

#### 2. Group rows into workouts using WorkoutId column

In [76]:
def get_workouts(topsets):
    # Get all unique workout_ids (No duplicates)
    workout_ids = set([t['WorkoutId'] for t in topsets])

    # Group topsets into workouts
    workouts = []
    for workout_id in workout_ids:
        topsets_in_workout = [t for t in topsets if t['WorkoutId'] == workout_id]
        workouts.append({
            'WorkoutId': workout_id,
            'StartDate': topsets_in_workout[0]['StartDate'],
            'TopSets': [{"TopSetId": t['TopSetId'], "ExcerciseId": t['ExcerciseId'], "ExerciseName": t['ExerciseName'], "Weight": t['Weight'], "Repetitions": t['Repetitions'] } for t in topsets_in_workout]
        })
    return workouts

In [77]:
print(json.dumps(get_workouts(topsets), indent=2))

[
  {
    "WorkoutId": 1,
    "StartDate": "2022-06-29 00:00:00.000",
    "TopSets": [
      {
        "TopSetId": 1,
        "ExcerciseId": 2,
        "ExerciseName": "Bench",
        "Weight": 40,
        "Repetitions": 11
      }
    ]
  },
  {
    "WorkoutId": 2,
    "StartDate": "2022-07-07 00:00:00.000",
    "TopSets": [
      {
        "TopSetId": 2,
        "ExcerciseId": 1,
        "ExerciseName": "Squat",
        "Weight": 65,
        "Repetitions": 5
      },
      {
        "TopSetId": 3,
        "ExcerciseId": 4,
        "ExerciseName": "Hotep",
        "Weight": 30,
        "Repetitions": 6
      }
    ]
  },
  {
    "WorkoutId": 3,
    "StartDate": "2022-07-12 00:00:00.000",
    "TopSets": [
      {
        "TopSetId": 4,
        "ExcerciseId": 2,
        "ExerciseName": "Bench",
        "Weight": 60,
        "Repetitions": 4
      },
      {
        "TopSetId": 5,
        "ExcerciseId": 3,
        "ExerciseName": "Deadlift",
        "Weight": 100,
        "Repetitions":

In [73]:
def get_all_exercises_from_topsets(topsets):
    exercise_ids = set([t['ExcerciseId'] for t in topsets])
    exercises = []
    for exercise_id in exercise_ids:
        exercises.append({
            'ExcerciseId': exercise_id,
            'ExerciseName': next((t['ExerciseName'] for t in topsets if t['ExcerciseId'] == exercise_id), 'Unknown')
        })
    return exercises

get_all_exercises_from_topsets(topsets)


[{'ExcerciseId': 1, 'ExerciseName': 'Squat'},
 {'ExcerciseId': 2, 'ExerciseName': 'Bench'},
 {'ExcerciseId': 3, 'ExerciseName': 'Deadlift'},
 {'ExcerciseId': 4, 'ExerciseName': 'Hotep'}]

### 3. Compose functions together to acheive final model

In [81]:
def convert_to_view_model(topsets):
    return {
        'PersonId': next((t['PersonId'] for t in topsets), -1),
        'PersonName': next((t['PersonName'] for t in topsets), -1),
        'Exercises': get_all_exercises_from_topsets(topsets),
        'Workouts': get_workouts(topsets)
    }

In [82]:
print(json.dumps(convert_to_view_model(topsets), indent=2))

{
  "PersonId": 1,
  "PersonName": "Gabe",
  "Exercises": [
    {
      "ExcerciseId": 1,
      "ExerciseName": "Squat"
    },
    {
      "ExcerciseId": 2,
      "ExerciseName": "Bench"
    },
    {
      "ExcerciseId": 3,
      "ExerciseName": "Deadlift"
    },
    {
      "ExcerciseId": 4,
      "ExerciseName": "Hotep"
    }
  ],
  "Workouts": [
    {
      "WorkoutId": 1,
      "StartDate": "2022-06-29 00:00:00.000",
      "TopSets": [
        {
          "TopSetId": 1,
          "ExcerciseId": 2,
          "ExerciseName": "Bench",
          "Weight": 40,
          "Repetitions": 11
        }
      ]
    },
    {
      "WorkoutId": 2,
      "StartDate": "2022-07-07 00:00:00.000",
      "TopSets": [
        {
          "TopSetId": 2,
          "ExcerciseId": 1,
          "ExerciseName": "Squat",
          "Weight": 65,
          "Repetitions": 5
        },
        {
          "TopSetId": 3,
          "ExcerciseId": 4,
          "ExerciseName": "Hotep",
          "Weight": 30,
      